In [1]:
from pyspark.sql.functions import col

StatementMeta(sparkpool, 2, 2, Finished, Available, Finished)

In [2]:
# Leyendo archivo parquet

df = spark.read.format('parquet').load('abfss://refined@adlsproyectos.dfs.core.windows.net/ExtNew/3304BD0B-96AC-419B-BEDE-E96D4AD45CE6_15_0-1.parquet')

StatementMeta(sparkpool, 2, 3, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(sparkpool, 2, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4f4ba611-5843-4f87-b13f-9b0bf5d51e99)

In [4]:
# Obtener todas las columnas en una variable

AllColumns = df.schema.names
print(AllColumns)

StatementMeta(sparkpool, 2, 5, Finished, Available, Finished)

['Line Number', 'Year', 'Month', 'State', 'Labor Force', 'Employed', 'Unemployed', 'Unemployment Rate', 'Industry', 'Gender', 'Education Level', 'Date Inserted', 'Aggregation Level', 'Data Accuracy']


In [5]:
# Cantidad de nulls por campo

for column in AllColumns:
    nullCount = df.filter(df[column].isNull()).count()
    if(nullCount > 0):
        print('Numero de nulls en '+ column + ': ' + str(nullCount))

StatementMeta(sparkpool, 2, 6, Finished, Available, Finished)

Numero de nulls en Line Number: 20
Numero de nulls en Year: 20
Numero de nulls en Month: 20
Numero de nulls en State: 20
Numero de nulls en Labor Force: 20
Numero de nulls en Employed: 20
Numero de nulls en Unemployed: 20
Numero de nulls en Unemployment Rate: 20
Numero de nulls en Industry: 20
Numero de nulls en Gender: 20
Numero de nulls en Education Level: 20
Numero de nulls en Date Inserted: 20
Numero de nulls en Aggregation Level: 65
Numero de nulls en Data Accuracy: 77


In [6]:
# Encontrar el conteo de filas duplicadas usando groupBy

df.groupBy('Line Number','Year').count().filter(col('count')>1).show()

StatementMeta(sparkpool, 2, 7, Finished, Available, Finished)

+-----------+----+-----+
|Line Number|Year|count|
+-----------+----+-----+
|       null|null|   20|
+-----------+----+-----+



In [7]:
# Visualizar los valores null para los campos 'Line Number' y 'Year'

df.filter((col('Line Number').isNull() & (col('Year').isNull()))).show()

StatementMeta(sparkpool, 2, 8, Finished, Available, Finished)

+-----------+----+-----+-----+-----------+--------+----------+-----------------+--------+------+---------------+-------------+-----------------+-------------+
|Line Number|Year|Month|State|Labor Force|Employed|Unemployed|Unemployment Rate|Industry|Gender|Education Level|Date Inserted|Aggregation Level|Data Accuracy|
+-----------+----+-----+-----+-----------+--------+----------+-----------------+--------+------+---------------+-------------+-----------------+-------------+
|       null|null| null| null|       null|    null|      null|             null|    null|  null|           null|         null|             null|         null|
|       null|null| null| null|       null|    null|      null|             null|    null|  null|           null|         null|             null|         null|
|       null|null| null| null|       null|    null|      null|             null|    null|  null|           null|         null|             null|         null|
|       null|null| null| null|       null|    

In [8]:
# Usar Dropna 
# La fila se elimina sólo si los valores de todos los campos son null o NaN

df_all = df.dropna(how='all')

StatementMeta(sparkpool, 2, 9, Finished, Available, Finished)

In [9]:
# Visualizar los valores null para los campos 'Line Number' y 'Year'

df_all.filter((col('Line Number').isNull() & (col('Year').isNull()))).show()

StatementMeta(sparkpool, 2, 10, Finished, Available, Finished)

+-----------+----+-----+-----+-----------+--------+----------+-----------------+--------+------+---------------+-------------+-----------------+-------------+
|Line Number|Year|Month|State|Labor Force|Employed|Unemployed|Unemployment Rate|Industry|Gender|Education Level|Date Inserted|Aggregation Level|Data Accuracy|
+-----------+----+-----+-----+-----------+--------+----------+-----------------+--------+------+---------------+-------------+-----------------+-------------+
+-----------+----+-----+-----+-----------+--------+----------+-----------------+--------+------+---------------+-------------+-----------------+-------------+



In [10]:
# Cantidad de nulls por campo

for column in AllColumns:
    nullCount = df_all.filter(df[column].isNull()).count()
    if(nullCount > 0):
        print('Numero de nulls en '+ column + ': ' + str(nullCount))

StatementMeta(sparkpool, 2, 11, Finished, Available, Finished)

Numero de nulls en Aggregation Level: 45
Numero de nulls en Data Accuracy: 57


In [ ]:
# Reemplazando los valores null por 'unknown' en todas las columnas
# Este valor solo será reemplazado para los campos con tipo de dato STRING

df_clean = df_all.fillna('unknown')

StatementMeta(sparkpool, 2, 12, Finished, Available, Finished)

In [12]:
# Cantidad de nulls por campo

for column in AllColumns:
    nullCount = df_clean.filter(df_clean[column].isNull()).count()
    print('Numero de nulls en '+ column + ': ' + str(nullCount))

StatementMeta(sparkpool, 2, 13, Finished, Available, Finished)

Numero de nulls en Line Number: 0
Numero de nulls en Year: 0
Numero de nulls en Month: 0
Numero de nulls en State: 0
Numero de nulls en Labor Force: 0
Numero de nulls en Employed: 0
Numero de nulls en Unemployed: 0
Numero de nulls en Unemployment Rate: 0
Numero de nulls en Industry: 0
Numero de nulls en Gender: 0
Numero de nulls en Education Level: 0
Numero de nulls en Date Inserted: 0
Numero de nulls en Aggregation Level: 0
Numero de nulls en Data Accuracy: 0


In [13]:
# Escribiendo los datos en una ubicación

df_clean.write.format('parquet')\
             .mode('overwrite')\
             .save('abfss://refined@adlsproyectos.dfs.core.windows.net/NoNulls/')

StatementMeta(sparkpool, 2, 14, Finished, Available, Finished)